* [x] Get data into csv to work in pandas
* [ ] Perform sentiment classification with Hugging Face model (2024 SOTA)
* [ ] Extract from text key words that resulted in sentiment class
* [ ] topic modeling?
* [ ] other advanced analytics for reviews using LLM/HuggingFace
* [ ] compare competition side-by-side (Japanese restaurants)
* [ ] get criteria for ranking/qualities in restaurant based on aggregation of common words through clustering

In [1]:
!pip install duckdb jupysql -q transformers --quiet 

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
import duckdb

%load_ext sql
conn = duckdb.connect()
%sql conn --alias duckdb

import re
from transformers import pipeline, AutoTokenizer

# from transformers import 

Deploy Streamlit apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


2024-05-16 21:16:15.463592: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-16 21:16:15.463809: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-16 21:16:15.657395: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
df = pd.read_csv('/kaggle/input/yelp-revised/yelp_biz_rev.csv')
df.head()

/tmp/ipykernel_18/1107626245.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/kaggle/input/yelp-revised/yelp_biz_rev.csv')


,Unnamed: 0,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories,review_id,user_id,review_stars,text,date
0,0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80.0,"{'key': ['RestaurantsDelivery', 'OutdoorSeating', 'BusinessAcceptsCreditCards', 'BusinessParking', 'BikeParking', 'RestaurantsPriceRange2', 'RestaurantsTakeOut', 'ByAppointmentOnly', 'WiFi', 'Alcohol', 'Caters'], 'value': ['False', 'False', 'False', ""{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}"", 'True', '1', 'True', 'False', ""u'free'"", ""u'none'"", 'True']}","Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries",BXQcBN0iAi1lAUxibGLFzA,6_SpY41LIHZuIaiDs5FMKA,4.0,"This is nice little Chinese bakery in the heart of Philadelphia's Chinatown! The female cashier was very friendly (flirtatious!) and the pastries shown in nicely adorned display cases. I stopped by early one evening had a sesame ball, which was filled with bean paste. The glutinous rice of the ball was nicely flavored, similar to Bai Tang Gao. Definitely as place worth stopping at if you are in the area.",2014-05-26 01:09:53
1,1,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80.0,"{'key': ['RestaurantsDelivery', 'OutdoorSeating', 'BusinessAcceptsCreditCards', 'BusinessParking', 'BikeParking', 'RestaurantsPriceRange2', 'RestaurantsTakeOut', 'ByAppointmentOnly', 'WiFi', 'Alcohol', 'Caters'], 'value': ['False', 'False', 'False', ""{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}"", 'True', '1', 'True', 'False', ""u'free'"", ""u'none'"", 'True']}","Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries",uduvUCvi9w3T2bSGivCfXg,tCXElwhzekJEH6QJe3xs7Q,4.0,"This is the bakery I usually go to in Chinatown. They have a decent variety of buns and pastries. Red bean-filled buns, mung bean-filled buns, coconut stuff, egg tarts etc. If you buy a dozen, they will give you two more for free. The prices are standard for a Chinese bakery, with most things at approximately $1. The service has always been good. They do serve coffee and tea. There are a couple of tables inside, with usually the expected group of older Chinese guys chillin' in the corner table debating about something. Great place to stock up on the buns, which I usually do before leaving town.",2013-10-05 15:19:06
2,2,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80.0,"{'key': ['RestaurantsDelivery', 'OutdoorSeating', 'BusinessAcceptsCreditCards', 'BusinessParking', 'BikeParking', 'RestaurantsPriceRange2', 'RestaurantsTakeOut', 'ByAppointmentOnly', 'WiFi', 'Alcohol', 'Caters'], 'value': ['False', 'False', 'False', ""{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}"", 'True', '1', 'True', 'False', ""u'free'"", ""u'none'"", 'True']}","Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries",a0vwPOqDXXZuJkbBW2356g,WqfKtI-aGMmvbA9pPUxNQQ,5.0,"A delightful find in Chinatown! Very clean, and very kind service. Everyone always seems happy in here, and their treats are absolutely delicious. You can also get a ton of sweet treats without any judgement! They're my favorite.",2013-10-25 01:34:57
3,3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80.0,"{'key': ['RestaurantsDelivery', 'OutdoorSeating', 'BusinessAcceptsCreditCards', 'BusinessParking', 'BikeParking', 'RestaurantsPriceRange2', 'RestaurantsTakeOut', 'ByAppointmentOnly', 'WiFi', 'Alcohol', 'Caters'], 'value': ['False', 'False', 'False', ""{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}"", 'True', '1', 'True', 'False', ""u'free'"", ""u'none'"", 'True']}","Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries",MKNp_CdR2k2202-c8GN5Dw,3-1va0IQfK-9tUMzfHWfTA,5.0,I ordered a graduation cake for my niece and 

In [4]:
query = """
select review_id
, text
from df
"""

df_sent2 = duckdb.query(query).df()
df_sent2

,review_id,text
0,BXQcBN0iAi1lAUxibGLFzA,"This is nice little Chinese bakery in the heart of Philadelphia's Chinatown! The female cashier was very friendly (flirtatious!) and the pastries shown in nicely adorned display cases. I stopped by early one evening had a sesame ball, which was filled with bean paste. The glutinous rice of the ball was nicely flavored, similar to Bai Tang Gao. Definitely as place worth stopping at if you are in the area."
1,uduvUCvi9w3T2bSGivCfXg,"This is the bakery I usually go to in Chinatown. They have a decent variety of buns and pastries. Red bean-filled buns, mung bean-filled buns, coconut stuff, egg tarts etc. If you buy a dozen, they will give you two more for free. The prices are standard for a Chinese bakery, with most things at approximately $1. The service has always been good. They do serve coffee and tea. There are a couple of tables inside, with usually the expected group of older Chinese guys chillin' in the corner table debating about something. Great place to stock up on the buns, which I usually do before leaving town."
2,a0vwPOqDXXZuJkbBW2356g,"A delightful find in Chinatown! Very clean, and very kind service. Everyone always seems happy in here, and their treats are absolutely delicious. You can also get a ton of sweet treats without any judgement! They're my favorite."
3,MKNp_CdR2k2202-c8GN5Dw,I ordered a graduation cake for my niece and it came out absolutely perfect. Will come again for future events! Staff was nice also and the cake was really good.
4,D1GisLDPe84Rrk_R4X2brQ,"HK-STYLE MILK TEA: FOUR STARS\n\nNot quite sure why there are two Saint Honore Pastries business listings on Yelp, but I have been checking in to this one.\n\nBoyfriend and I go by it all the time while in Chinatown, but it never crossed our mind to try it. I think we might have tried it once a year or two ago, but it wasn't memorable. I decided to try the HK-style milk tea not too long ago and was hooked. I think I went here almost every day one week to get the milk tea, only to realize that I should REALLY watch my sugar intake...it's sooo good! \n\nI personally don't think it's better than Zhong Gang's HK-style milk tea because they have their off days when a certain lady makes them (not enough ice, too much ice, etc.).\n\nOn a side note: I wanted to try the crispy roast pork bun, but accidentally said the wrong name in Chinese and got the roast pork bun. It was decent enough, although I will get to trying the crispy roast pork bun soon! \n\nI will be back~"
...,...,...
3773767,Kt3gFeW1rhZz7RuiV-6Tcw,"This is my favorite food truck! I only wish I lived closer to Philly so that I could eat here more often. The prices are affordable, the portions are generous, the staff is friendly, and the food is AMAZING. \n\nThe last time I went to the Plum Pit, I ordered a jerk chicken sandwich to split with a friend. Learn from our mistake--order your own sandwich, and if you can't finish it, save half for later. We ended up going back for another sandwich (the one with chicken tenders, mac & cheese, and bacon--I can't remember the name) anyway. We also got two orders of bread pudding to bring home, which, though pretty good cold, is ever better when you heat it up in the oven for 10-15 minutes."
3773768,ruy3Ycey_gGbwkE_3TX1Fg,"This food truck was stupid. Stupidly delicious. \n\nShould be a 4.5 because they were a little slow compared to other trucks, but once the food showed up, totally ON POINT.\n\nWe had the bacon Mac and cheese with the French fry bowl, topped with jerk chicken. I could not get enough of the jerk chicken. And the bacon Mac and cheese was super creamy ( again 4.5 we saw a big can of nacho cheese instead of good cheese "" . \n\nWould definitely eat here again if they were at the Stables."
3773769,C_l8NTpvNOEUorEmEOusaA,Bubba never disappoints i go to his fb page and hunt him down and now i have my friends and family stalking him too. One of the main reasons is the surf and turf sandwich if its not o

In [5]:
# reduce file size by getting sample
#sample 500,000 to use less RAM
sample = df_sent2.sample(n=500000)
len(sample)

500000

In [6]:
sample.head(1)

,review_id,text
1573710,g2gZdyQuf5vtqYd24PVzgQ,"We made reservations and our table wasn't ready when we arrived. We had to wait an additional fifteen minutes, so why did I make reservations? When the restaurant called to confirm, I requested a table by the railings and they said no problem. Sadly, they didn't have a table reserved for us but had tables reserved for two big parties that were an hour ahead of us so they didn't want to sit us by the railings because of the two big parties. The manager of the hostesses came out to tell us more excuses and only wanted to shake my husbands hand (big mistake being she was a female as well) after apologizing for the confusion. This restaurant really needs to rethink their process. The stupid hostess stand is in the middle of the restaurant and one side of the patio has a consistent line of people. Not a great ambiance for a restaurant at la encantada. Or I would have to say, the lines are due to poor management, probably because the hostess manager doesn't know what she's doing. The food is ok....step up from chipotle I would say. After being treated badly and the restaurant not committed to our reservation time, I don't think we will be back. Tucson has too many good restaurants to settle on this expensive chipotle with rude managers."


# Text Pre-Processing

In [7]:
# text column needs to be changed to string...
sample['text'] = sample['text'].apply(str)

In [8]:
# Get code to clean text of HTML tags, punctuation?, emoji-like, slang converter (''n'-> ing) or get model that understands yum/yummy
def cleanhtml(x): 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', x) 
    return cleantext.replace('\n\n', ' ').replace('\n', '')
    
sample['clean_text'] = sample.apply(lambda row: cleanhtml(row['text']), axis=1)

# Examine sample of dirty comments to see if translation went through
sample

,review_id,text,clean_text
1573710,g2gZdyQuf5vtqYd24PVzgQ,"We made reservations and our table wasn't ready when we arrived. We had to wait an additional fifteen minutes, so why did I make reservations? When the restaurant called to confirm, I requested a table by the railings and they said no problem. Sadly, they didn't have a table reserved for us but had tables reserved for two big parties that were an hour ahead of us so they didn't want to sit us by the railings because of the two big parties. The manager of the hostesses came out to tell us more excuses and only wanted to shake my husbands hand (big mistake being she was a female as well) after apologizing for the confusion. This restaurant really needs to rethink their process. The stupid hostess stand is in the middle of the restaurant and one side of the patio has a consistent line of people. Not a great ambiance for a restaurant at la encantada. Or I would have to say, the lines are due to poor management, probably because the hostess manager doesn't know what she's doing. The food is ok....step up from chipotle I would say. After being treated badly and the restaurant not committed to our reservation time, I don't think we will be back. Tucson has too many good restaurants to settle on this expensive chipotle with rude managers.","We made reservations and our table wasn't ready when we arrived. We had to wait an additional fifteen minutes, so why did I make reservations? When the restaurant called to confirm, I requested a table by the railings and they said no problem. Sadly, they didn't have a table reserved for us but had tables reserved for two big parties that were an hour ahead of us so they didn't want to sit us by the railings because of the two big parties. The manager of the hostesses came out to tell us more excuses and only wanted to shake my husbands hand (big mistake being she was a female as well) after apologizing for the confusion. This restaurant really needs to rethink their process. The stupid hostess stand is in the middle of the restaurant and one side of the patio has a consistent line of people. Not a great ambiance for a restaurant at la encantada. Or I would have to say, the lines are due to poor management, probably because the hostess manager doesn't know what she's doing. The food is ok....step up from chipotle I would say. After being treated badly and the restaurant not committed to our reservation time, I don't think we will be back. Tucson has too many good restaurants to settle on this expensive chipotle with rude managers."
1175984,6qcMZa7AP_3dCMz9KOfOZw,"Great place.... pizza is top shelf .... place is nice and clean, my go to place for good quality food now... thnx","Great place.... pizza is top shelf .... place is nice and clean, my go to place for good quality food now... thnx"
3444675,yzrXlx7aEA_ctH7uyAtkog,"Great local Old Seminole Heights Business! The menu is really great and the atmosphere is chill for me. I've been here several times, and never disappointed! Get your ass in here and partake!","Great local Old Seminole Heights Business! The menu is really great and the atmosphere is chill for me. I've been here several times, and never disappointed! Get your ass in here and partake!"
1033901,wMzqAqA_qLefuUuiTLCkpQ,"Went there last Saturday night for a bday party for my gf. Unfortunately, you cannot reserve a lane ahead of time, you have to go there and put your name on the waiting list and wait until a lane is available. I called around 4pm and asked about the wait for a 7pm start time, and I was told ""get here as soon as you can"" and was there around 5pm (from talking to the staff the normal wait time for a lane on Saturday is 1.5-2 hours). They really should consider taking reservations with a credit card so you don't have to sit around for hours waiting for a lane to open up. Another thing, they only allow 7 people to bowl per lane, so if you have more than 7 people in your party, and you go on a Saturday night, you're

In [9]:
# normalize text by lowering it all
sample['clean_text'] = sample['clean_text'].str.lower()

In [10]:
%%time
# Due to transformer error:
# If you are running a transformer model with HuggingFace, there is a chance that one of the input sentences is longer than 512 tokens. Either truncate or split your sentences. I suspect the shorter sentences are padded to 512 tokens.

#1. get token length/total words with max: 500 total words as max length since tokenizer can add suffixes as a token
#2. eliminate tokens lengths greater than max/summarize text using chatgpt?

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# sentence = "Sentence to check whether it exceeds 512 tokens"

# tokenized_sentence = tokenizer.tokenize(sentence)

# print(len(sentence.split()))  # here is the default length of the sentence
# print(len(tokenized_sentence)) # here is the tokenized length

## Example applied to df

sample['num_tokens_tok'] = sample.apply(lambda row: len(tokenizer.tokenize(row.clean_text)), axis = 1)
sample.head()

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (548 > 512). Running this sequence through the model will result in indexing errors


CPU times: user 6min 17s, sys: 100 ms, total: 6min 17s
Wall time: 6min 18s


,review_id,text,clean_text,num_tokens_tok
1573710,g2gZdyQuf5vtqYd24PVzgQ,"We made reservations and our table wasn't ready when we arrived. We had to wait an additional fifteen minutes, so why did I make reservations? When the restaurant called to confirm, I requested a table by the railings and they said no problem. Sadly, they didn't have a table reserved for us but had tables reserved for two big parties that were an hour ahead of us so they didn't want to sit us by the railings because of the two big parties. The manager of the hostesses came out to tell us more excuses and only wanted to shake my husbands hand (big mistake being she was a female as well) after apologizing for the confusion. This restaurant really needs to rethink their process. The stupid hostess stand is in the middle of the restaurant and one side of the patio has a consistent line of people. Not a great ambiance for a restaurant at la encantada. Or I would have to say, the lines are due to poor management, probably because the hostess manager doesn't know what she's doing. The food is ok....step up from chipotle I would say. After being treated badly and the restaurant not committed to our reservation time, I don't think we will be back. Tucson has too many good restaurants to settle on this expensive chipotle with rude managers.","we made reservations and our table wasn't ready when we arrived. we had to wait an additional fifteen minutes, so why did i make reservations? when the restaurant called to confirm, i requested a table by the railings and they said no problem. sadly, they didn't have a table reserved for us but had tables reserved for two big parties that were an hour ahead of us so they didn't want to sit us by the railings because of the two big parties. the manager of the hostesses came out to tell us more excuses and only wanted to shake my husbands hand (big mistake being she was a female as well) after apologizing for the confusion. this restaurant really needs to rethink their process. the stupid hostess stand is in the middle of the restaurant and one side of the patio has a consistent line of people. not a great ambiance for a restaurant at la encantada. or i would have to say, the lines are due to poor management, probably because the hostess manager doesn't know what she's doing. the food is ok....step up from chipotle i would say. after being treated badly and the restaurant not committed to our reservation time, i don't think we will be back. tucson has too many good restaurants to settle on this expensive chipotle with rude managers.",279
1175984,6qcMZa7AP_3dCMz9KOfOZw,"Great place.... pizza is top shelf .... place is nice and clean, my go to place for good quality food now... thnx","great place.... pizza is top shelf .... place is nice and clean, my go to place for good quality food now... thnx",34
3444675,yzrXlx7aEA_ctH7uyAtkog,"Great local Old Seminole Heights Business! The menu is really great and the atmosphere is chill for me. I've been here several times, and never disappointed! Get your ass in here and partake!","great local old seminole heights business! the menu is really great and the atmosphere is chill for me. i've been here several times, and never disappointed! get your ass in here and partake!",41
1033901,wMzqAqA_qLefuUuiTLCkpQ,"Went there last Saturday night for a bday party for my gf. Unfortunately, you cannot reserve a lane ahead of time, you have to go there and put your name on the waiting list and wait until a lane is available. I called around 4pm and asked about the wait for a 7pm start time, and I was told ""get here as soon as you can"" and was there around 5pm (from talking to the staff the normal wait time for a lane on Saturday is 1.5-2 hours). They really should consider taking reservations with a credit card so you don't have to sit around for hours waiting for a lane to open up. Another thing, they only allow 7 people to bowl per lane, so if you have more than 7 people in your party, and you go on

In [11]:
# get rid of rows that are <450 words in length since this tokenizer is approx of tokenizer used for sentiment model
query = """
select *
from sample
where num_tokens_tok < 450
"""

sample_short_rev = duckdb.query(query).df()
sample_short_rev.head()

,review_id,text,clean_text,num_tokens_tok
0,g2gZdyQuf5vtqYd24PVzgQ,"We made reservations and our table wasn't ready when we arrived. We had to wait an additional fifteen minutes, so why did I make reservations? When the restaurant called to confirm, I requested a table by the railings and they said no problem. Sadly, they didn't have a table reserved for us but had tables reserved for two big parties that were an hour ahead of us so they didn't want to sit us by the railings because of the two big parties. The manager of the hostesses came out to tell us more excuses and only wanted to shake my husbands hand (big mistake being she was a female as well) after apologizing for the confusion. This restaurant really needs to rethink their process. The stupid hostess stand is in the middle of the restaurant and one side of the patio has a consistent line of people. Not a great ambiance for a restaurant at la encantada. Or I would have to say, the lines are due to poor management, probably because the hostess manager doesn't know what she's doing. The food is ok....step up from chipotle I would say. After being treated badly and the restaurant not committed to our reservation time, I don't think we will be back. Tucson has too many good restaurants to settle on this expensive chipotle with rude managers.","we made reservations and our table wasn't ready when we arrived. we had to wait an additional fifteen minutes, so why did i make reservations? when the restaurant called to confirm, i requested a table by the railings and they said no problem. sadly, they didn't have a table reserved for us but had tables reserved for two big parties that were an hour ahead of us so they didn't want to sit us by the railings because of the two big parties. the manager of the hostesses came out to tell us more excuses and only wanted to shake my husbands hand (big mistake being she was a female as well) after apologizing for the confusion. this restaurant really needs to rethink their process. the stupid hostess stand is in the middle of the restaurant and one side of the patio has a consistent line of people. not a great ambiance for a restaurant at la encantada. or i would have to say, the lines are due to poor management, probably because the hostess manager doesn't know what she's doing. the food is ok....step up from chipotle i would say. after being treated badly and the restaurant not committed to our reservation time, i don't think we will be back. tucson has too many good restaurants to settle on this expensive chipotle with rude managers.",279
1,6qcMZa7AP_3dCMz9KOfOZw,"Great place.... pizza is top shelf .... place is nice and clean, my go to place for good quality food now... thnx","great place.... pizza is top shelf .... place is nice and clean, my go to place for good quality food now... thnx",34
2,yzrXlx7aEA_ctH7uyAtkog,"Great local Old Seminole Heights Business! The menu is really great and the atmosphere is chill for me. I've been here several times, and never disappointed! Get your ass in here and partake!","great local old seminole heights business! the menu is really great and the atmosphere is chill for me. i've been here several times, and never disappointed! get your ass in here and partake!",41
3,S68fDYnXD-EKj5gr4b9G_g,"We hosted my dad's funeral luncheon at Tre Fratelli on a Thursday afternoon at 1 pm. After a sad day we all needed a chance to rest and visit with one another and enjoy some good food. I was so impressed with The staffthat I knew it was going to be the perfect spot. We hosted 51 people total and the food was absolutely divine. (The chicken Parmesan was my favorite.) The private room was beautiful and the service was excellent. I won't share what we paid on here because I feel like we got an excellent deal. We also got to bring in our own beer/wine which helped cut down on costs. Now I have a dining room table full of food they sent me home with. Thank you. You gave us a wonderful lunch, and a great place to reme

In [12]:
#Check max number tokens
query = """
select max(num_tokens_tok) as max_tokens
from sample_short_rev
"""

duckdb.query(query).df()

,max_tokens
0,449


In [13]:
# remaining of sample after token length filter
len(sample_short_rev)

488741

# Sentiment Classification

In [14]:
# using Hugging Face sentiment classifier trained on Yelp restaurant reviews

# Set up the inference pipeline using a model from the 🤗 Hub

sentiment_analysis = pipeline(model="mrcaelumn/yelp_restaurant_review_sentiment_analysis")

config.json:   0%|          | 0.00/769 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/315 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [15]:
test = sample_short_rev.sample(n=5000)

In [16]:
# %%time

# sample_short_rev = (
# sample_short_rev.assign(sentiment = lambda x: x['clean_text'].apply(lambda s: sentiment_analysis(s)))
#     .assign(
#          label = lambda x: x['sentiment'].apply(lambda s: (s[0]['label'])),
#          score = lambda x: x['sentiment'].apply(lambda s: (s[0]['score']))
#     )
# )

# sample_short_rev.head()

In [17]:
%%time

test = (
test.assign(sentiment = lambda x: x['clean_text'].apply(lambda s: sentiment_analysis(s)))
    .assign(
         label = lambda x: x['sentiment'].apply(lambda s: (s[0]['label'])),
         score = lambda x: x['sentiment'].apply(lambda s: (s[0]['score']))
    )
)

test.head()

CPU times: user 21min 45s, sys: 3.21 s, total: 21min 48s
Wall time: 10min 56s


,review_id,text,clean_text,num_tokens_tok,sentiment,label,score
439855,qreBJKGM7GjeC7mNtarT_w,"I really wanted to like this place because they had chill outdoor space, and decent space inside as well but they just could not get service together for brunch. Missed orders, missing items from other orders and what happened to be the sausage debacle of 2018 as they kept bringing out pieces of ground beef without any flavors, seasoning or casing and claiming that that was how they made sausages. \n\nThe best was when they brought out two hamburger patties at one point saying THOSE were their sausage patties... Ummm it's okay to say you dont have it on the menu, like we get it. We laughed about it after the fact but definitely one of the worst brunch experiences I've had in Philly. With the plethora of brunch spots elsewhere I would skip over this one. Maybe they have a better bar scene....who knows","i really wanted to like this place because they had chill outdoor space, and decent space inside as well but they just could not get service together for brunch. missed orders, missing items from other orders and what happened to be the sausage debacle of 2018 as they kept bringing out pieces of ground beef without any flavors, seasoning or casing and claiming that that was how they made sausages. the best was when they brought out two hamburger patties at one point saying those were their sausage patties... ummm it's okay to say you dont have it on the menu, like we get it. we laughed about it after the fact but definitely one of the worst brunch experiences i've had in philly. with the plethora of brunch spots elsewhere i would skip over this one. maybe they have a better bar scene....who knows",184,"[{'label': 'LABEL_0', 'score': 0.9780673384666443}]",LABEL_0,0.978067
261023,9vXWgrnA8GN-hW0qH8NvNQ,Food very good; happy hour wine good too. Service just ok and watch their in-house advertising for happy hour....can be misleading and their customer service is poor in following up with complaints/concerns.,food very good; happy hour wine good too. service just ok and watch their in-house advertising for happy hour....can be misleading and their customer service is poor in following up with complaints/concerns.,44,"[{'label': 'LABEL_1', 'score': 0.8368664383888245}]",LABEL_1,0.836866
462449,kEn69ivUYCMBce1pPuv-JA,"It's a really great location and fantastic prices! The people who work there are really nice but I have gotten my order mixed up a couple times.. You can't really hold it against them, they're only human!","it's a really great location and fantastic prices! the people who work there are really nice but i have gotten my order mixed up a couple times.. you can't really hold it against them, they're only human!",48,"[{'label': 'LABEL_2', 'score': 0.9810840487480164}]",LABEL_2,0.981084
39136,iEC-4VcZMZsuI3X926GJgQ,I didn't know that mankind had achieved deliciousness such as this. Horrible wait and fend for yourself service pretty much sums it up here in the French quarter though.,i didn't know that mankind had achieved deliciousness such as this. horrible wait and fend for yourself service pretty much sums it up here in the french quarter though.,35,"[{'label': 'LABEL_0', 'score': 0.6745988130569458}]",LABEL_0,0.674599
209475,0LLliilOUheUq-ZKi1hOiQ,Got a very tasty but also slightly moldy Mushroom Sandwich. I've eaten here a couple times before and the food has been okay.,got a very tasty but also slightly moldy mushroom sandwich. i've eaten here a couple times before and the food has been okay.,29,"[{'label': 'LABEL_1', 'score': 0.8897585272789001}]",LABEL_1,0.889759


In [18]:
test['label'].value_counts()

label
LABEL_2    3551
LABEL_0     939
LABEL_1     510
Name: count, dtype: int64